Correlation Table for S&P 500?

## Stock Price prediction based on the usage of RNN | LSTM

Predicting the close stock price

In [ ]:
#!python3.9 -m pip install --upgrade mplfinance
#!python3.9 -m pip --upgrade pandas
# !python3.9 -m pip install pandas-datareader --upgrade
# !python3.9 -m pip install yfinance
# !python3.9 -m pip install fredapi

In [1]:
import math
import pandas_datareader.data as web
import numpy as np
import pandas as pd
import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
# from mplfinance import candlestick_ohlc

from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mdates
plt.style.use('fivethirtyeight')

from pandas_datareader import data as pdr
import yfinance as yfin

from config import fred_api


2023-02-14 20:57:32.140427: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Fill in the stock you want to crawl:

In [2]:
company = 'amc'
provider = 'yahoo'

In [3]:
# Get stock quote
#start = datetime.datetime(2018-3-1) # Change Date if needed
start = datetime.datetime(2018, 1, 1) # Change Date if needed

end = datetime.datetime.now()

In [4]:
# df = web.DataReader(company, provider, start, end)

yfin.pdr_override()

df = pdr.get_data_yahoo(company, start, end)

[*********************100%***********************]  1 of 1 completed


In [5]:
df = df.reset_index(level=0)

In [6]:
from fredapi import Fred
fred = Fred(api_key=fred_api)

# sp500 = fred.get_series('SP500')
# gdp = fred.get_series_latest_release('GDP')

# Retrieve interest rate data from FRED
interest_rates = fred.get_series('FEDFUNDS', observation_start=start)
empl = fred.get_series('PAYEMS', observation_start=start) # seasonally ajusted non-farm employment in thousands, All Employees, Total Nonfarm (PAYEMS)
gdp = fred.get_series('GDPC1', observation_start=start) # Seasonally adjusted real GDP 
unemp =fred.get_series('UNRATE', observation_start=start) # seasonally ajusted unemployment rate in percent
SP500=fred.get_series('SP500', observation_start=start)

# Convert index to datetime
interest_rates.index = pd.to_datetime(interest_rates.index)

df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

df['Interest_Rate'] = interest_rates.reindex(df.index, method='nearest')
# Compute rolling average of interest rates
interest_rates_ma = interest_rates.rolling(window=20).mean()

# Combine interest rate data with stock price data
df['InterestRates_calc'] = interest_rates_ma.reindex(df.index, method='nearest')
df['InterestRates'] = interest_rates_ma.reindex(df.index, method='nearest')

In [7]:
df['SP500'] = SP500.reindex(df.index, method='nearest')
df['unemployment_rate'] = unemp.reindex(df.index, method='nearest')
df['gdp'] = gdp.reindex(df.index, method='nearest')
df['Nonfarm'] = empl.reindex(df.index, method='nearest') # Units:  Thousands of Persons, Seasonally Adjusted

In [8]:
df = df.reset_index(level=0)
df

,Date,Open,High,Low,Close,Adj Close,Volume,Interest_Rate,InterestRates_calc,InterestRates,SP500,unemployment_rate,gdp,Nonfarm
0,2018-01-02,15.20,15.65,14.90,15.400,12.549153,2257200,1.41,NaN,NaN,2695.81,4.0,18437.127,147670.0
1,2018-01-03,15.45,15.50,14.90,15.000,12.223199,1631000,1.41,NaN,NaN,2713.06,4.0,18437.127,147670.0
2,2018-01-04,15.00,15.10,14.35,14.450,11.775018,2141000,1.41,NaN,NaN,2723.99,4.0,18437.127,147670.0
3,2018-01-05,14.50,14.56,14.05,14.100,11.489810,2009000,1.41,NaN,NaN,2743.15,4.0,18437.127,147670.0
4,2018-01-08,14.30,14.35,13.65,13.800,11.245344,2347400,1.41,NaN,NaN,2747.71,4.0,18437.127,147670.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1284,2023-02-08,6.12,6.13,5.61,5.720,5.720000,31424400,4.33,1.256,1.256,4117.86,3.4,20198.091,155073.0
1285,2023-02-09,5.87,5.88,5.14,5.360,5.360000,29113300,4.33,1.256,1.256,4081.50,3.4,20198.091,155073.0
1286,2023-02-10,5.21,5.21,4.64,4.900,4.900000,38589100,4.33,1.256,1.256,4090.46,3.4,20198.091,155073.0
1287,2023-02-13,4.75,4.91,4.58,4.680,4.680000,26219400,4.33,1.256,1.256,4137.29,3.4,20198.091,155073.0


In [ ]:
df['70ma'] = df['Adj Close'].rolling(window=70).mean()
df['20ma_adj'] = df['Adj Close'].rolling(window=20).mean()
df.dropna(inplace=True)

# Calculate daily price change
df['price_change'] = df['High'] - df['Low']

# Calculate 20-day moving average
df['20ma_close'] = df['Close'].rolling(window=20).mean()

# Calculate Bollinger Bands
df['std'] = df['Close'].rolling(window=20).std()
df['upper_band'] = df['20ma_close'] + 2 * df['std']
df['lower_band'] = df['20ma_close'] - 2 * df['std']

df['upper_band'] = df['20ma_adj'] + 2 * df['std']
df['lower_band'] = df['20ma_adj'] - 2 * df['std']


In [ ]:
df.shape

In [ ]:
df.tail()

In [ ]:
df_ohlc = df['Adj Close'].resample('10D').ohlc() # open high low close
df_ohlc.dropna(inplace=True)
df_vol = df['Volume'].resample('10D').sum()

In [ ]:
df_ohlc.tail(5)

In [ ]:
df_ohlc.reset_index(inplace=True)

In [ ]:
df_ohlc['Date'] = df_ohlc['Date'].map(mdates.date2num)

In [ ]:
ax1 = plt.subplot2grid((6,1), (0,0), rowspan = 5, colspan = 1)
ax2 = plt.subplot2grid((6,1), (5,0), rowspan = 1, colspan = 1, sharex=ax1)
ax2.ticklabel_format(style='plain')

ax1.xaxis_date()
candlestick_ohlc(ax1, df_ohlc.values, width=2, colorup='g')
ax2.fill_between(df_vol.index.map(mdates.date2num), df_vol.values,0)

## Train the model

In [ ]:
df.head(2)

In [ ]:
df.columns

In [ ]:
# Create new df w/ "Close" col

df2 = df.filter(['Adj Close'])
# convert to numpy array
dataset = df2.values

# Get no of rows to train the (LSTM) model on
# train it on 80 % of the data 
training_data_len = math.ceil(len(dataset) * .8)  # math.ceil -> rounding
training_data_len

In [ ]:
pd.set_option('display.max_columns', None)
df.tail(2)

In [ ]:
# Scale the data

scaler = MinMaxScaler(feature_range=(0,1))  # values / features ranges 0-1 inclusive

# Transformed data
scaled_data = scaler.fit_transform(dataset.reshape(-1,1))
# scaled_data

In [ ]:
# Create the training data set
# Create scaled training data set
# train_data = scaled_data[0:training_data_len, :]

# Split data into x_train & y_train data set
#reate empty lists
x_train = [] # independent variables / features
y_train = [] # dependent / target variable

prediction_days = 180 # 60 days, modify as needed

for i in range(prediction_days, len(scaled_data)):  
    x_train.append(scaled_data[i - prediction_days:i, 0]) # never reaches i, not including i
    y_train.append(scaled_data[i, 0])
    # if i  <= 60:
        #print(x_train)
        #print()
        #print(y_train)
        #print()

In [ ]:
# Convert x_train & y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
x_train.shape

In [ ]:
# Reshape data, LSTM expects input to be 3-dimensional
# x_train = np.shape(x_train, (943, 60, 1))
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [ ]:
# Build LSTM model
# This will be the model architecture
from keras.layers import Dropout


model = Sequential()
model.add(LSTM(20, return_sequences=True, input_shape=(x_train.shape[1], 1)))
#model.add(Dropout(.2))
model.add(LSTM(20, return_sequences=True))
#model.add(Dropout(.2))
model.add(LSTM(20, return_sequences=False))
#model.add(Dense(25))
model.add(Dense(1))

# the number of hidden units should be in-between the number of input units and output classes

In [ ]:
# Compile model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

When training a neural network for stock price prediction, mean_squared_error (MSE) is a common loss function to use. This is because the goal of stock price prediction is to minimize the difference between the predicted and actual prices.

MSE measures the average squared difference between the predicted and actual values. It is a good choice for regression problems where the output values are continuous. 

In [ ]:
# Train model
#history = model.fit(x_train, y_train, epochs=10, batch_size=5) #validation_split=0.33  , verbose=0

from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

history = model.fit(x_train, y_train, epochs=100, batch_size=5, validation_split=0.33, callbacks=[early_stop]) #verbose=0,



In [ ]:
# Define the size of the training set
training_data_len = int(len(dataset) * 0.67)

# Create testing dataset
test_data = scaled_data[training_data_len - prediction_days:]

# Create data set x_test & y_test
x_test = []
y_test = dataset[training_data_len:]

for i in range(prediction_days, test_data.shape[0]):
    x_test.append(test_data[i - prediction_days:i, 0])

# Convert x_test and y_test to numpy arrays
x_test = np.array(x_test)
y_test = np.array(y_test)

# Reshape x_test to 3 dimensions for input to LSTM
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))


In [ ]:
# Create testing dataset
# Create new array containing scaled values from 900 - 960
# test_data = scaled_data[training_data_len - prediction_days: , :]

# Create data set x_test & y_test
# x_test = []
# y_test = dataset[training_data_len: , :]  # all of values that we want the model to predict, the actual test values
# for i in range(prediction_days, len(test_data)):
#     x_test.append(test_data[i- prediction_days:i, 0])


# Convert data to numpy array
# x_test = np.array(x_test)

# Reshape data -> 3-dimensional shape
# x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))   # x_test.shape[0] -> number of samples ; x_test.shape[1] -> number of timesteps; number of features: 1 (close price)


In [ ]:
# Get the models predicted price values

predictions = scaler.inverse_transform(model.predict(x_test))
# we want predictions to contain same values as y_test data set
 # unscale values

In [ ]:
# Get root mean square error (RSME)
rmse = np.sqrt(np.mean(((predictions- y_test)**2)))

In [ ]:
print(rmse)

### Plotting 

In [ ]:
# Plot data
train = df[:training_data_len]
valid = df[training_data_len:]
valid['Predictions'] = predictions

# Viz
plt.figure(figsize=(14,6))
plt.title('Model')
plt.xlabel('Date', fontsize=11)
plt.ylabel('Close Price US $', fontsize=11)
plt.plot(train['Adj Close'])
plt.plot(valid[['Adj Close', 'Predictions']])
plt.legend(['Train', 'Validation', 'Predictions'], loc='lower left')

blue = data the model was trained on
red = actual values / actual closing price for the dates
orange = what model predicted to be

In [ ]:
# Show valid and predicted price
valid.tail()

# Predict close price for future

In [ ]:
df_future = df.filter(['Close'])
last_60_days = df_future[-prediction_days:].values
last_60_days_scaled = scaler.transform(last_60_days)
X_test = []
X_test.append(last_60_days_scaled)
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

pred_price = model.predict(X_test)
pred_price = scaler.inverse_transform(pred_price)
print(pred_price)

In [ ]:
df_future = df.filter(['Close'])
last_60_days = df_future[-prediction_days-3:].values
last_60_days_scaled = scaler.transform(last_60_days)
X_test = []
X_test.append(last_60_days_scaled)
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

pred_price = model.predict(X_test)
pred_price = scaler.inverse_transform(pred_price)
print(f'Prediction: {pred_price}')


In [ ]:
# Prediction: [[283.78705]]


--------------------
--------------------

## Different approach

In [ ]:
'''test_start = datetime.datetime(2022,1,1)
test_end = datetime.datetime.now()

test_df = web.DataReader(company, 'yahoo', test_start, test_end)
actual_prices = test_df['Close'].values

total_dataset = pd.concat((df['Close'], test_df['Close']), axis=0)

model_inputs = total_dataset[len(total_dataset) - len(test_df) - prediction_days:].values
model_inputs = model_inputs.reshape(-1,1)
model_inputs = scaler.transform(model_inputs)'''

In [ ]:
# Make predictions on test data

In [ ]:
x_test = []

for i in range(prediction_days, len(model_inputs)):
  x_test.append(model_inputs[i-prediction_days:i,0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

predicted_prices = model.predict(x_test)
predicted_prices = scaler.inverse_transform(predicted_prices)

In [ ]:
# Plot the test pred
plt.figure(figsize=(14,6))
plt.plot(actual_prices, color='black', label=f'Actual {company} Price')
plt.plot(predicted_prices, color='green', label=f'Predicted {company} Price')
plt.title(f'{company} Share Price')
plt.xlabel('Time')
plt.ylabel(f'{company} Share Price in US $')
plt.legend(loc='upper left')

In [ ]:
# Predict next day

real_data = [model_inputs[len(model_inputs) + 1 - prediction_days:len(model_inputs + 1), 0]]
real_data = np.array(real_data)
real_data = np.reshape(real_data, (real_data.shape[0], real_data.shape[1],1))

# print(scaler.inverse_transform(real_data[-1]))

prediction = model.predict(real_data)
prediction = scaler.inverse_transform(prediction)
print(f'Prediction: {prediction}')


Different approach w/ avg of highest and lowest price



In [ ]:
# First calculate the mid prices from the highest and lowest
high_prices = df.loc[:,'High'].as_matrix()
low_prices = df.loc[:,'Low'].as_matrix()
mid_prices = (high_prices+low_prices)/2.0

In [ ]:
train_data = mid_prices[:11000]
test_data = mid_prices[11000:]

In [ ]:
scaler = MinMaxScaler()
train_data = train_data.reshape(-1,1)
test_data = test_data.reshape(-1,1)

In [ ]:
# Train the Scaler with training data and smooth data
smoothing_window_size = 2500
for di in range(0,10000,smoothing_window_size):
    scaler.fit(train_data[di:di+smoothing_window_size,:])
    train_data[di:di+smoothing_window_size,:] = scaler.transform(train_data[di:di+smoothing_window_size,:])

# You normalize the last bit of remaining data
scaler.fit(train_data[di+smoothing_window_size:,:])
train_data[di+smoothing_window_size:,:] = scaler.transform(train_data[di+smoothing_window_size:,:])

In [ ]:
train_data = train_data.reshape(-1)

# Normalize test data
test_data = scaler.transform(test_data).reshape(-1)

In [ ]:
EMA = 0.0
gamma = 0.1
for ti in range(11000):
  EMA = gamma*train_data[ti] + (1-gamma)*EMA
  train_data[ti] = EMA

# Used for visualization and test purposes
all_mid_data = np.concatenate([train_data,test_data],axis=0)

In [ ]:
window_size = 100
N = train_data.size
std_avg_predictions = []
std_avg_x = []
mse_errors = []

for pred_idx in range(window_size,N):

    if pred_idx >= N:
        date = dt.datetime.strptime(k, '%Y-%m-%d').date() + dt.timedelta(days=1)
    else:
        date = df.loc[pred_idx,'Date']

    std_avg_predictions.append(np.mean(train_data[pred_idx-window_size:pred_idx]))
    mse_errors.append((std_avg_predictions[-1]-train_data[pred_idx])**2)
    std_avg_x.append(date)

print('MSE error for standard averaging: %.5f'%(0.5*np.mean(mse_errors)))

In [ ]:
plt.figure(figsize = (18,9))
plt.plot(range(df.shape[0]),all_mid_data,color='b',label='True')
plt.plot(range(window_size,N),std_avg_predictions,color='orange',label='Prediction')
#plt.xticks(range(0,df.shape[0],50),df['Date'].loc[::50],rotation=45)
plt.xlabel('Date')
plt.ylabel('Mid Price')
plt.legend(fontsize=18)
plt.show()

In [ ]:
window_size = 100
N = train_data.size

run_avg_predictions = []
run_avg_x = []

mse_errors = []

running_mean = 0.0
run_avg_predictions.append(running_mean)

decay = 0.5

for pred_idx in range(1,N):

    running_mean = running_mean*decay + (1.0-decay)*train_data[pred_idx-1]
    run_avg_predictions.append(running_mean)
    mse_errors.append((run_avg_predictions[-1]-train_data[pred_idx])**2)
    run_avg_x.append(date)

print('MSE error for EMA averaging: %.5f'%(0.5*np.mean(mse_errors)))


In [ ]:

plt.figure(figsize = (18,9))
plt.plot(range(df.shape[0]),all_mid_data,color='b',label='True')
plt.plot(range(0,N),run_avg_predictions,color='orange', label='Prediction')
#plt.xticks(range(0,df.shape[0],50),df['Date'].loc[::50],rotation=45)
plt.xlabel('Date')
plt.ylabel('Mid Price')
plt.legend(fontsize=18)
plt.show()
